In [ ]:
import os
os.chdir('..')

In [ ]:
from sim_circuit import QuantumRegister, QuantumCircuit
from util import all_close

def geometric_sequence_circuit(n, theta):

    N = 2**n

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n): # <1>
        qc.h(q[j])
        qc.p(2 ** j * theta, q[j])

    assert all_close(qc.run(), [1/sqrt(N)*cis(k*theta) for k in range(N)]) # <2>

In [ ]:
from math import pi

def encode_frequency(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(2 * pi / 2 ** (n - j) * v, q[j]) # <1>

    qc.report('geometric_sequence')

    qc.iqft(range(n))

    qc.report('iqft')

    return qc

In [ ]:
from util import print_state_table

n = 3
v = 4.7

state = encode_frequency(n, v).run()

In [ ]:
print_state_table(state)

In [ ]:
def complex_sincd(n, v):
    N = 2**n
    return [prod(cos((v-k)*pi/2**(m+1)) for m in range(n))*cis((N-1)/N * (v-k)*pi) for k in range(2**n)]

In [ ]:
def complex_sincd_combined_cis(n, v):
    N = 2**n
    return [prod(cos((v-k)*pi/2**(m+1)) for m in range(n))*cis((N-1)/N * (v-k)*pi) for k in range(2**n)]

In [ ]:
def complex_sincd_sum(n, v):
    # same as value encoding state
    N = 2**n
    return [2/N*sum(cos((2*m+1)*(v-k)*pi/N) for m in range(N//2))*cis((N-1) * (v-k)*pi/N) for k in range(N)]


def test_sum_form():
    n = 3
    v = 4.7
    assert all_close(complex_sincd(n, v), complex_sincd_sum(n, v))

In [ ]:
def recursive_complex_discrete_sinc(n, v):
    if n == 1:
        return [cis(v*pi/2)*cos(v*pi/2), -1j*cis(v*pi/2)*sin(v*pi/2)]

    a = recursive_complex_discrete_sinc(n-1, v)

    return ([a[k]*cis((v - k)*pi/2**n)*cos((v - k)*pi/2**n) for k in range(2**(n-1))] +
            [a[k]*cis((v - k)*pi/2**n)*-1j*sin((v - k)*pi/2**n) for k in range(2**(n-1))])

In [ ]:
def geom_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(pi * 2 ** -j * v, q[j])

    return qc

In [ ]:
def encode_frequency_q_alt(n, v):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])
        qc.p(pi * 2 ** -j * v, q[j])

    qc.report('signal')

    qc.iqft(range(n)[::-1], swap=False)

    qc.report('iqft')

    return qc

In [ ]:
from util import is_close

def test_discrete_sinc_by_digit():
    n = 3
    v = 4.7

    for l in range(2**n): # <1>
        s = bin(l)[2:].zfill(n)
        assert(len(s) == n)
        p = 1
        k = 0
        for m in range(n): # <2>
            if s[m] == '0':
                p *= cos((v - k)*pi/2**(m+1))**2
            else:
                p *= sin((v - k)*pi/2**(m+1))**2
                k += 2**m

        assert isclose(p, prod(cos((v-k)*pi/2**(j+1)) for j in range(n))**2) # <3>

In [ ]:
def recursive_discrete_sinc(n, v):
    if n == 1:
        return [cos(v*pi/2)**2, sin(v*pi/2)**2]

    p = recursive_discrete_sinc(n-1, v)

    return [p[k]*cos((v - k)*pi/2**n)**2 for k in range(2**(n-1))] + [p[k]*sin((v - k)*pi/2**n)**2 for k in range(2**(n-1))]

In [ ]:
def discrete_sinc_coin_flips(n, v, count=10000):
    samples = []
    for _ in range(count):
        k = 0
        for m in range(n):
            flip = np.random.binomial(1, sin((v-k)*pi/2**(m+1))**2)
            k += flip*2**m

        samples.append(k)

    return samples

In [ ]:
def raised_cosine(n, mu):
    N = 2 ** n
    assert (0 <= mu < 2 ** n)

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.h(q[n - 1])
    qc.p(-pi * mu / N * 2, q[n - 1])

    qc.report('fourier_coefficients')

    qc.qft(q[::-1], False) # <1>

    qc.report('qft')

    return qc

In [ ]:
qc = raised_cosine(3, 3.25)
state = qc.run()

In [ ]:
print_state_table(state)

In [ ]:
from math import sqrt, cos
from util import cis

N = 8
mu = 3.25
a = [sqrt(2/N) * cos((k - mu)*pi/N) * cis((k-mu)*pi/N) for k in range(N)]
assert all_close(state, a)

In [ ]:
s = N / 2
p = [1 / (2 * s) * (1 + cos((x - mu) / s * pi)) for x in range(N)]
p1 = [1 / s * cos((x - mu) / (2 * s) * pi) ** 2 for x in range(N)]

probs = [2/N*(cos((k - mu)*pi/N))**2 for k in range(N)] 

assert all_close(p, probs)
assert all_close(p1, probs)

In [ ]:
from math import acos

def sin_4(n):
    theta = acos(sqrt(2 / 3))
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    qc.ry(2 * theta, q[n - 1])
    qc.p(pi, q[n - 1])
    qc.cry(pi / 2, q[n - 1], q[0])

    for i in range(1, n - 1):
        qc.cx(q[0], q[i])

    qc.report('frequencies')

    qc.qft(q[::-1], False)

    qc.report('qft')

    return qc

In [ ]:
n = 3           
N = 2 ** n      
qc = sin_4(n)   
state = qc.run()

In [ ]:
print_state_table(state)

In [ ]:
from math import sin

s = [sqrt(8 / (3 * N)) * (sin(k * pi / N)) ** 2 for k in range(N)]
assert all_close(state, s)
p = [8 / 3 / N * (sin(k * pi / N)) ** 4 for k in range(N)]  # alternatively: 8/3/N*(cos((k - N/2)*pi/N))**4
assert all_close([abs(state[k])**2 for k in range(N)], p)

In [ ]:
theta = -pi/2         
q = QuantumRegister(n)
qc = QuantumCircuit(q)
qc.ry(theta, q[0])    
qc.report('state')

In [ ]:
s = [cos(k*pi/2 - theta/2) if k in [0, 1] else 0 for k in range(N)]
assert all_close(qc.reports['state'][2], s)

In [ ]:
qc.qft(q)       
qc.report('qft')

In [ ]:
s = [1/sqrt(N)*cos(theta/2) + 1/sqrt(N)*sin(theta/2)*cis(k*2*pi/N) for k in range(N)]
assert all_close(qc.reports['qft'][2], s)

In [ ]:
def one_over_sine(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(1, n):
        qc.h(i)

    qc.x(0)
    qc.iqft(q[::-1], swap=False)

    return qc

In [ ]:
qc = one_over_sine(4)
state = qc.run()

In [ ]:
print_state_table(state)

In [ ]:
from math import tan
n = 4
N = 2**n

qc = one_over_sine(n)
state = qc.run()

assert is_close(abs(state[0])**2, 1/2)

for k in range(1, N):
    if k%2 == 1:
        assert is_close(abs(state[k])**2, (sqrt(2)/N/sin(k*pi/N))**2)
        assert all_close([state[k]], [sqrt(2)/N*(-1 + 1j*1/tan(k*pi/N))])
    else:
        assert is_close(abs(state[k])**2, 0)